In [1]:
!pip install -q nltk evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.8 MB/s eta 0:00:00


In [2]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from evaluate import load
from jiwer import wer, cer
import json
import numpy as np

nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ic_results_path = '/content/drive/MyDrive/Imagenes Proyecto/results_ic_git.json'
ocr_results_path = '/content/drive/MyDrive/Imagenes Proyecto/results_ocr_git.json'

ic_output_path = '/content/drive/MyDrive/Imagenes Proyecto/metrics_ic_git.json'
ocr_output_path = '/content/drive/MyDrive/Imagenes Proyecto/metrics_ocr_git.json'

with open(ic_results_path, 'r', encoding='utf-8') as f:
    ic_results = json.load(f)

with open(ocr_results_path, 'r', encoding='utf-8') as f:
    ocr_results = json.load(f)

print(f"IC: {len(ic_results['results'])} imágenes")
print(f"OCR: {len(ocr_results['results'])} imágenes\n")

IC: 15 imágenes
OCR: 15 imágenes



In [4]:
def calculate_bleu(reference, hypothesis):
    """Calcula BLEU score"""
    ref_tokens = [reference.lower().split()]
    hyp_tokens = hypothesis.lower().split()
    smooth = SmoothingFunction().method1
    return sentence_bleu(ref_tokens, hyp_tokens, smoothing_function=smooth)

def calculate_meteor(reference, hypothesis):
    """Calcula METEOR score"""
    meteor = load('meteor')
    score = meteor.compute(predictions=[hypothesis], references=[[reference]])
    return score['meteor']

In [5]:
def calculate_cer(reference, hypothesis):
    """Calcula Character Error Rate (CER)"""
    if not reference or not hypothesis:
        return 1.0  # Error máximo si alguno está vacío
    return cer(reference, hypothesis)

def calculate_wer(reference, hypothesis):
    """Calcula Word Error Rate (WER)"""
    if not reference or not hypothesis:
        return 1.0  # Error máximo si alguno está vacío
    return wer(reference, hypothesis)


In [6]:
print("="*70)
print("CALCULANDO MÉTRICAS DE IMAGE CAPTIONING (IC)")
print("="*70 + "\n")

ic_metrics = []
ic_by_level = {
    "basic": {"bleu": [], "meteor": []},
    "intermediate": {"bleu": [], "meteor": []},
    "advanced": {"bleu": [], "meteor": []}
}

for image_name, image_data in ic_results['results'].items():
    print(f" {image_name}")

    manual_ic = image_data.get("manual_ic", {})
    prompt_outputs = image_data.get("prompt_outputs", [])

    image_metrics = {
        "image": image_name,
        "levels": {}
    }

    for output_data in prompt_outputs:
        level = output_data.get("level", "")
        generated = output_data.get("output", "")
        reference = manual_ic.get(level, "")

        if not reference or not generated:
            continue

        try:
            bleu = calculate_bleu(reference, generated)
            meteor = calculate_meteor(reference, generated)

            image_metrics["levels"][level] = {
                "reference": reference,
                "generated": generated,
                "bleu": round(bleu, 4),
                "meteor": round(meteor, 4)
            }

            ic_by_level[level]["bleu"].append(bleu)
            ic_by_level[level]["meteor"].append(meteor)

            print(f"  {level:12} | BLEU: {bleu:.4f} | METEOR: {meteor:.4f}")

        except Exception as e:
            print(f" Error en {level}: {e}")

    ic_metrics.append(image_metrics)
    print()

# Calcular promedios IC
print("="*70)
print("PROMEDIOS IC POR NIVEL")
print("="*70 + "\n")

ic_summary = {
    "model": ic_results.get("model", ""),
    "task": "Image_Captioning",
    "total_images": len(ic_results['results']),
    "metrics_by_level": {},
    "overall": {}
}

for level in ["basic", "intermediate", "advanced"]:
    bleu_scores = ic_by_level[level]["bleu"]
    meteor_scores = ic_by_level[level]["meteor"]

    if bleu_scores:
        ic_summary["metrics_by_level"][level] = {
            "bleu": {
                "mean": round(np.mean(bleu_scores), 4),
                "std": round(np.std(bleu_scores), 4),
                "min": round(min(bleu_scores), 4),
                "max": round(max(bleu_scores), 4)
            },
            "meteor": {
                "mean": round(np.mean(meteor_scores), 4),
                "std": round(np.std(meteor_scores), 4),
                "min": round(min(meteor_scores), 4),
                "max": round(max(meteor_scores), 4)
            },
            "samples": len(bleu_scores)
        }

        print(f"{level.upper()}")
        print(f"   BLEU:   {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f}")
        print(f"   METEOR: {np.mean(meteor_scores):.4f} ± {np.std(meteor_scores):.4f}\n")

# Promedio general IC
all_bleu = []
all_meteor = []
for level_data in ic_by_level.values():
    all_bleu.extend(level_data["bleu"])
    all_meteor.extend(level_data["meteor"])

ic_summary["overall"] = {
    "bleu": {
        "mean": round(np.mean(all_bleu), 4),
        "std": round(np.std(all_bleu), 4)
    },
    "meteor": {
        "mean": round(np.mean(all_meteor), 4),
        "std": round(np.std(all_meteor), 4)
    },
    "total_samples": len(all_bleu)
}

print(" PROMEDIO GENERAL IC")
print(f"BLEU:   {np.mean(all_bleu):.4f} ± {np.std(all_bleu):.4f}")
print(f"METEOR: {np.mean(all_meteor):.4f} ± {np.std(all_meteor):.4f}\n")


CALCULANDO MÉTRICAS DE IMAGE CAPTIONING (IC)

 01.jpg


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  basic        | BLEU: 0.0339 | METEOR: 0.4133


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


...

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  basic        | BLEU: 0.0540 | METEOR: 0.4689


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  intermediate | BLEU: 0.0025 | METEOR: 0.0791
  advanced     | BLEU: 0.0000 | METEOR: 0.0515

PROMEDIOS IC POR NIVEL

BASIC
   BLEU:   0.0536 ± 0.0556
   METEOR: 0.3443 ± 0.1732

INTERMEDIATE
   BLEU:   0.0059 ± 0.0033
   METEOR: 0.1052 ± 0.0463

ADVANCED
   BLEU:   0.0002 ± 0.0006
   METEOR: 0.0497 ± 0.0157

 PROMEDIO GENERAL IC
BLEU:   0.0199 ± 0.0401
METEOR: 0.1664 ± 0.1647



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
print("="*70)
print("CALCULANDO MÉTRICAS DE OCR")
print("="*70 + "\n")

ocr_metrics = []
ocr_scores = {"cer": [], "wer": []}

for image_name, image_data in ocr_results['results'].items():
    print(f" {image_name}")

    manual_ocr = image_data.get("manual_ocr", [])
    prompt_outputs = image_data.get("prompt_outputs", [])

    # Convertir lista de anotaciones a texto único
    if isinstance(manual_ocr, list):
        reference = " ".join(manual_ocr)
    else:
        reference = str(manual_ocr)

    image_metrics = {
        "image": image_name,
        "reference": reference,
        "prompts": []
    }

    for output_data in prompt_outputs:
        prompt = output_data.get("prompt", "")
        generated = output_data.get("output", "")

        if not reference or not generated:
            continue

        try:
            cer_score = calculate_cer(reference, generated)
            wer_score = calculate_wer(reference, generated)

            image_metrics["prompts"].append({
                "prompt": prompt,
                "generated": generated,
                "cer": round(cer_score, 4),
                "wer": round(wer_score, 4)
            })

            ocr_scores["cer"].append(cer_score)
            ocr_scores["wer"].append(wer_score)

            print(f"  '{prompt[:30]}...' | CER: {cer_score:.4f} | WER: {wer_score:.4f}")

        except Exception as e:
            print(f"  Error: {e}")

    ocr_metrics.append(image_metrics)
    print()

# Calcular promedios OCR
print("="*70)
print("PROMEDIOS OCR")
print("="*70 + "\n")

ocr_summary = {
    "model": ocr_results.get("model", ""),
    "task": "OCR",
    "total_images": len(ocr_results['results']),
    "metrics": {
        "cer": {
            "mean": round(np.mean(ocr_scores["cer"]), 4),
            "std": round(np.std(ocr_scores["cer"]), 4),
            "min": round(min(ocr_scores["cer"]), 4),
            "max": round(max(ocr_scores["cer"]), 4)
        },
        "wer": {
            "mean": round(np.mean(ocr_scores["wer"]), 4),
            "std": round(np.std(ocr_scores["wer"]), 4),
            "min": round(min(ocr_scores["wer"]), 4),
            "max": round(max(ocr_scores["wer"]), 4)
        }
    },
    "total_samples": len(ocr_scores["cer"])
}

print(f" CER:  {np.mean(ocr_scores['cer']):.4f} ± {np.std(ocr_scores['cer']):.4f}")
print(f"   Rango: [{min(ocr_scores['cer']):.4f}, {max(ocr_scores['cer']):.4f}]")
print(f"\n WER:  {np.mean(ocr_scores['wer']):.4f} ± {np.std(ocr_scores['wer']):.4f}")
print(f"   Rango: [{min(ocr_scores['wer']):.4f}, {max(ocr_scores['wer']):.4f}]")
print(f"\nTotal muestras: {len(ocr_scores['cer'])}\n")

CALCULANDO MÉTRICAS DE OCR

 20220101_035959.jpg
  '...' | CER: 0.8898 | WER: 1.0000
  'the text says...' | CER: 0.8898 | WER: 1.0000

 20230214_122422.jpg
  '...' | CER: 0.7789 | WER: 1.0000
  'the text says...' | CER: 0.7789 | WER: 1.0000

 20230430_150424.jpg
  '...' | CER: 0.9397 | WER: 0.9759
  'the text says...' | CER: 0.9397 | WER: 0.9759

 20230511_133754.jpg
  '...' | CER: 0.9068 | WER: 0.9913
  'the text says...' | CER: 0.9068 | WER: 0.9913

 20230811_151805.jpg
  '...' | CER: 0.9205 | WER: 1.0000
  'the text says...' | CER: 0.9205 | WER: 1.0000

 20231108_090334.jpg
  '...' | CER: 0.9436 | WER: 0.9929
  'the text says...' | CER: 0.9436 | WER: 0.9929

 20240401_124930.jpg
  '...' | CER: 0.9348 | WER: 0.9911
  'the text says...' | CER: 0.9348 | WER: 0.9911

 20240603_120302.jpg
  '...' | CER: 0.9594 | WER: 1.0000
  'the text says...' | CER: 0.9594 | WER: 1.0000

 20250331_130243.jpg
  '...' | CER: 0.8889 | WER: 1.0000
  'the text says...' | CER: 0.8889 | WER: 1.0000

 20250819